In [21]:
!pip install pandas numpy scikit-learn tensorflow

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, load_model # type: ignore
from tensorflow.keras.layers import Dense # type: ignore
from tensorflow.keras.utils import to_categorical # type: ignore
import os

1. **Data Loading and Preprocessing**:
- The dataset is loaded from a CSV file.
    - The `RoadSurface` column is mapped to numerical values.
    - Features are selected for training the model.
    - Labels are converted to categorical one-hot encoding.
    - The data is split into training and testing sets.
    - Feature scaling is applied to standardize the data.

In [14]:
def get_data(path):
    try:
        # Load dataset
        df = pd.read_csv(path)
    except FileNotFoundError:
        return "File not found"
    except Exception as e:
        return f"An error occurred while loading the dataset: {e}"
    
    # Data preprocessing
    df["RoadSurface"] = df["RoadSurface"].map({'Poor': 0, 'Avg': 1, 'Good': 2})

    # Feature selection
    features = ["IDMachines", "PeopleAtwork", "StreetLights", "Accidents", "DamagedMovers", "StRoadLength", "RoadCurvature", "HPBends", "RoadType", "RoadWidth", "AvgSpeed", "AgeOfRoad"]
    X = df[features]
    y = df["RoadSurface"]

    # Convert labels to categorical one-hot encoding
    y = to_categorical(y, num_classes=3)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

    # Feature scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

2. **Neural Network Model**:
    - A neural network model is built using the `Sequential` API from TensorFlow Keras.
    - The model consists of multiple dense layers with ReLU activation functions and a softmax output layer.
    - The model is compiled with categorical cross-entropy loss and the Adam optimizer.
    - The model is trained on the training data with a validation split.

In [19]:
def train_nn():
    output = ""
    
    X_train, X_test, y_train, y_test = get_data('datasets/rmdataset.csv')
    # Build the neural network model
    model = Sequential()
    model.add(Dense(12, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(3, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=15, batch_size=10, validation_split=0.2)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Model accuracy on test data: {accuracy * 100:.2f}%\n")

    # Ensure the directory exists
    os.makedirs('models/nn', exist_ok=True)

    # Save the model and scaler
    model.save('models/nn/mlp_model.h5')
    print("Model trained and saved\n")

    return None


3. **Model Evaluation and Saving**:
    - The model is evaluated on the test data to determine its accuracy.
    - The trained model and the scaler used for feature scaling are saved to disk.

4. **Prediction**:
    - The notebook includes an example of how to use the trained model to make predictions on new data.

In [20]:
result = train_nn()

Epoch 1/15


C:\Users\abhin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5370 - loss: 0.9082 - val_accuracy: 0.6194 - val_loss: 0.7254
Epoch 2/15
214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6577 - loss: 0.6660 - val_accuracy: 0.9049 - val_loss: 0.5012
Epoch 3/15
214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9205 - loss: 0.4555 - val_accuracy: 0.9254 - val_loss: 0.4086
Epoch 4/15
214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9223 - loss: 0.3700 - val_accuracy: 0.9179 - val_loss: 0.3517
Epoch 5/15
214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9274 - loss: 0.3246 - val_accuracy: 0.9179 - val_loss: 0.3140
Epoch 6/15
214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9255 - loss: 0.3028 - val_accuracy: 0.9254 - val_loss: 0.2810
Epoch 7/15
214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9398 - loss: 0.2578 - val_accuracy: 0.9291 - val_loss: 0.2622
Epoch 8/15
214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9469 - loss: 0.2348 - val_accuracy: 0.9310 - val_

Model accuracy on test data: 92.68%

Model trained and saved

